In [1]:
import pandas as pd 
import yfinance as yf
from datetime import datetime, timedelta


In [2]:
SP500_Url = "https://en.wikipedia.org/wiki/Nasdaq-100"
data_table = pd.read_html(
    SP500_Url,
    storage_options={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                                   "(KHTML, like Gecko) Chrome/123.0 Safari/537.36"}
)
data_table

[                                     0                                      1
 0   Nasdaq-100 Index from 1985 to 2015     Nasdaq-100 Index from 1985 to 2015
 1                           Foundation         January 31, 1985; 40 years ago
 2                             Operator                           Nasdaq, Inc.
 3                            Exchanges                                 Nasdaq
 4                       Trading symbol                            ^NDX or NDX
 5                         Constituents                                    101
 6                                 Type                              Large-cap
 7                           Market cap  US$25.34 trillion (as of May 1, 2025)
 8                     Weighting method     Free-float capitalization-weighted
 9                      Related indices                   Nasdaq Financial-100
 10                             Website                  nasdaq.com/nasdaq-100,
    Category  All-Time Highs[8]         All-Time Hi

In [3]:
tickers = data_table[4]["Ticker"].tolist()

In [4]:
supp = ['ASML', 'AZN', 'F', 'G', 'GOOGL', 'H', 'I', 'J', 'L', 'O', 'Q', 'R', 'U', 'VRSK', 'W', 'Y']

tickers = [x for x in tickers if x not in supp]
print(len(tickers), len(supp))

97 16


In [5]:
# Extraction des données
Date = data_table[5]['Date']
Removed = data_table[5]['Removed']['Ticker']  # Supposant que 'Removed' est un DataFrame ou Series
Added = data_table[5]['Added']['Ticker']      # Supposant que 'Added' est un DataFrame ou Series

# Combinaison des données dans un DataFrame
Data_Change = pd.concat([Date, Added, Removed], axis = 1)

#Date au format Date
Data_Change['Date'] = pd.to_datetime(Data_Change['Date'])

# Changer le format de la date en 'jour/mois/année'
Data_Change['Date'] = Data_Change['Date'].dt.strftime('%Y-%m-%d')


# Renommer toutes les colonnes
Data_Change.columns = ['Date', 'Added', 'Removed']

# Mettre 'colonne_a' en tant qu'index
Data_Change.set_index('Date', inplace=True)

# Nan en vide
Data_Change = Data_Change.fillna('')

Data_Change

,Added,Removed
Date,,
2025-07-28,TRI,
2025-07-17,,ANSS
2025-05-19,SHOP,MDB
2024-12-23,PLTR,ILMN
2024-12-23,MSTR,MRNA
...,...,...
2007-07-12,FWLT,BMET
2007-06-01,CEPH,MEDI
2007-03-08,UAUA,AEOS


In [6]:
len(tickers)

97

In [7]:
# Supposons que la liste tickers ait déjà 503 éléments au départ
# tickers = ['AAPL', 'MSFT', ...]  # Exemples de tickers initiaux

# Initialiser un dictionnaire vide
dict = {}

# Boucle pour parcourir les données et mettre à jour le dictionnaire
for i in range(0, len(Data_Change)):
    
    # Ajouter les nouveaux tickers (si ce n'est pas déjà un ensemble vide)
    tickers.extend(Data_Change.Added[i])
    
    # Essayer de supprimer les tickers retirés, si possible
    try:
        tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés
    except ValueError:
        # Si l'élément à supprimer n'existe pas dans la liste, continuer sans erreur
        pass
    
    # Trier les tickers
    tickers = sorted(set(tickers))  # Utilisation de set pour éviter les doublons

    # Ajouter une **copie** de la liste `tickers` dans le dictionnaire
    dict[Data_Change.index[i]] = tickers.copy()

df = pd.DataFrame(list(dict.items()), columns=['date', 'tickers'])

df['date'] = pd.to_datetime(df['date'])

# Mettre la colonne 'date' comme index
df.set_index('date', inplace=True)

df

/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1477/1527443372.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers.extend(Data_Change.Added[i])
/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1477/1527443372.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés


,tickers
date,
2025-07-28,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [8]:
# Récupère l'index en liste
new_index = list(df.index)

# Remplace la première valeur
new_index[0] = datetime.today().date()

# Réapplique l'index modifié
df.index = pd.to_datetime(new_index)

In [9]:
df

,tickers
2025-10-07,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [10]:

data_SP500_old = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet')
last_date = data_SP500_old['Date'].sort_values(ascending=True).iloc[-1] + timedelta(days=1)
last_date


Timestamp('2025-10-04 00:00:00')

In [11]:

df['Last_Date'] = last_date
df

,tickers,Last_Date
2025-10-07,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-10-04
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-10-04
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-10-04
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-10-04
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-10-04
...,...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-10-04
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-10-04
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-10-04
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-10-04


In [12]:

# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': last_date, 'End': end_date})
resultat


/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_1477/2328952231.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())


,Start,End
tickers,,
AAPL,2025-10-04,2025-10-07
ABNB,2025-10-04,2025-10-07
ADBE,2025-10-04,2025-10-07
ADI,2025-10-04,2025-10-07
ADP,2025-10-04,2025-10-07
...,...,...
VRTX,2025-10-04,2025-10-07
WBD,2025-10-04,2025-10-07
WDAY,2025-10-04,2025-10-07


In [13]:
"""
# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
start_date = df.groupby('tickers').apply(lambda x: x.index.min())
end_date = df.groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': start_date, 'End': end_date})
resultat
"""

"\n# Initialiser une DataFrame vide pour stocker toutes les données\ndata_SP500 = pd.DataFrame()\n\ndf = df.explode('tickers')\nstart_date = df.groupby('tickers').apply(lambda x: x.index.min())\nend_date = df.groupby('tickers').apply(lambda x: x.index.max())\n\n# Combiner les résultats dans une nouvelle DataFrame\nresultat = pd.DataFrame({'Start': start_date, 'End': end_date})\nresultat\n"

In [14]:
# Télécharger les données pour chaque ticker
for ticker in resultat.index:
    start_date = resultat.loc[ticker, 'Start']
    end_date = resultat.loc[ticker, 'End']

    print(f"Téléchargement des données pour {ticker} de {start_date} à {end_date}")

    # Télécharger les données de l'action entre les dates spécifiées
    data = yf.download(
                        ticker,
                        start=start_date,
                        end=end_date,
                        interval="1d",
                        auto_adjust=True,
                        actions=False
                        )

    # Ajouter une colonne pour identifier le ticker
    data['Ticker'] = ticker

    # On retire le niveau des tickers pour ne garder que 'Close' et 'Volume'
    data.columns = data.columns.droplevel(level=1)
    
    # Concaténer les données dans la DataFrame globale
    data_SP500 = pd.concat([data_SP500, data])

# Affichage de la DataFrame résultante
data_SP500

Téléchargement des données pour AAPL de 2025-10-04 00:00:00 à 2025-10-07 00:00:00


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour ABNB de 2025-10-04 00:00:00 à 2025-10-07 00:00:00
Téléchargement des données pour ADBE de 2025-10-04 00:00:00 à 2025-10-07 00:00:00


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour ADI de 2025-10-04 00:00:00 à 2025-10-07 00:00:00
Téléchargement des données pour ADP de 2025-10-04 00:00:00 à 2025-10-07 00:00:00


...


[*********************100%***********************]  1 of 1 completed


Téléchargement des données pour WDAY de 2025-10-04 00:00:00 à 2025-10-07 00:00:00
Téléchargement des données pour XEL de 2025-10-04 00:00:00 à 2025-10-07 00:00:00


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Téléchargement des données pour ZS de 2025-10-04 00:00:00 à 2025-10-07 00:00:00


Price,Close,High,Low,Open,Volume,Ticker,Adj Close
Date,,,,,,,
2025-10-06,256.690002,259.070007,255.050003,257.989990,44292200.0,AAPL,NaN
2025-10-06,120.349998,123.150002,120.050003,120.730003,5469000.0,ABNB,NaN
2025-10-06,350.140015,357.339996,342.040009,346.929993,5326600.0,ADBE,NaN
2025-10-06,242.500000,245.009995,238.919998,245.009995,3863100.0,ADI,NaN
2025-10-06,290.549988,291.950012,288.489990,291.500000,1703700.0,ADP,NaN
...,...,...,...,...,...,...,...
2025-10-06,402.899994,405.450012,400.739990,404.660004,1191100.0,VRTX,NaN
2025-10-06,19.080000,19.180000,18.879999,19.030001,17518400.0,WBD,NaN
2025-10-06,238.360001,243.949997,234.690002,235.660004,2253000.0,WDAY,NaN


In [15]:
data_SP500 = data_SP500.reset_index()
data_SP500

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
0,2025-10-06,256.690002,259.070007,255.050003,257.989990,44292200.0,AAPL,NaN
1,2025-10-06,120.349998,123.150002,120.050003,120.730003,5469000.0,ABNB,NaN
2,2025-10-06,350.140015,357.339996,342.040009,346.929993,5326600.0,ADBE,NaN
3,2025-10-06,242.500000,245.009995,238.919998,245.009995,3863100.0,ADI,NaN
4,2025-10-06,290.549988,291.950012,288.489990,291.500000,1703700.0,ADP,NaN
...,...,...,...,...,...,...,...,...
94,2025-10-06,402.899994,405.450012,400.739990,404.660004,1191100.0,VRTX,NaN
95,2025-10-06,19.080000,19.180000,18.879999,19.030001,17518400.0,WBD,NaN
96,2025-10-06,238.360001,243.949997,234.690002,235.660004,2253000.0,WDAY,NaN
97,2025-10-06,81.000000,81.279999,79.889999,80.410004,4082600.0,XEL,NaN


In [16]:
df_concat = pd.concat([data_SP500, data_SP500_old])
df_concat

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
0,2025-10-06,256.690002,259.070007,255.050003,257.989990,44292200.0,AAPL,NaN
1,2025-10-06,120.349998,123.150002,120.050003,120.730003,5469000.0,ABNB,NaN
2,2025-10-06,350.140015,357.339996,342.040009,346.929993,5326600.0,ADBE,NaN
3,2025-10-06,242.500000,245.009995,238.919998,245.009995,3863100.0,ADI,NaN
4,2025-10-06,290.549988,291.950012,288.489990,291.500000,1703700.0,ADP,NaN
...,...,...,...,...,...,...,...,...
200,2025-10-03,187.619995,190.360001,185.380005,189.190002,137340500.0,NVDA,NaN
98,2025-10-03,271.220001,272.440002,268.140015,270.000000,2417200.0,DASH,NaN
197,2025-10-03,1153.319946,1168.000000,1143.219971,1165.000000,3133100.0,NFLX,NaN
59,2025-10-03,347.269989,353.149994,345.000000,347.239990,1056300.0,CDNS,NaN


In [17]:
df_concat.sort_values(by=['Date'], ascending=True, inplace=True)
df_concat.tail(50)

Price,Date,Close,High,Low,Open,Volume,Ticker,Adj Close
49,2025-10-06,25.299999,25.830000,25.190001,25.799999,23702900.0,KDP,NaN
51,2025-10-06,1139.709961,1152.119995,1121.680054,1121.680054,946000.0,KLAC,NaN
77,2025-10-06,71.290001,71.870003,69.452003,70.220001,16586300.0,PYPL,NaN
78,2025-10-06,168.619995,170.880005,167.720001,170.539993,8262100.0,QCOM,NaN
79,2025-10-06,194.220001,195.479996,189.960007,192.919998,297300.0,R,NaN
80,2025-10-06,585.479980,603.650024,584.409973,602.000000,1071900.0,REGN,NaN
81,2025-10-06,506.989990,507.359985,497.119995,501.899994,1210200.0,ROP,NaN
82,2025-10-06,152.259995,153.460007,151.199997,152.240005,2669700.0,ROST,NaN
83,2025-10-06,82.110001,86.290001,82.059998,86.290001,14978700.0,SBUX,NaN
84,2025-10-06,164.500000,169.690002,162.809998,166.550003,11311800.0,SHOP,NaN


In [18]:
df_concat['Date'].max()

Timestamp('2025-10-06 00:00:00')

In [19]:
# Sauvegarder la DataFrame résultante dans un fichier CSV
#data_SP500.to_csv('/Users/forget/PyCharmMiscProject/historical_data_S&P500_From_Wiki.csv')

In [20]:
"""
mask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]
mask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]
mask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]
"""

"\nmask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]\nmask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]\nmask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]\n"

In [21]:
"""
idx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)
data_SP500 = data_SP500.drop(index=idx_to_drop).copy()
"""

'\nidx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)\ndata_SP500 = data_SP500.drop(index=idx_to_drop).copy()\n'

In [22]:
df_concat.to_parquet("/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet")